## **Import Libraries**

In [1]:
import numpy as np
import requests
import pandas as pd
from io import StringIO
import warnings
warnings.filterwarnings('ignore')
import random
random.seed(6)
#import catboost 
#from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder, RobustScaler
from sklearn.model_selection import KFold,GroupKFold, RepeatedKFold, GridSearchCV, cross_validate, train_test_split
from sklearn.metrics import confusion_matrix, precision_score, accuracy_score, recall_score, roc_auc_score, log_loss
from numpy import mean, std
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
pd.set_option('display.max_colwidth', 250)
pd.set_option('display.max_columns', 500)
pd.options.mode.chained_assignment = None 
import xgboost
from xgboost import XGBClassifier
from sklearn.utils import shuffle


## **LOAD DATA FROM MY GOOGLE DRIVE FOLDER**

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ss = pd.read_csv('/content/drive/My Drive/laduma/SampleSubmission.csv')
vd = pd.read_csv('/content/drive/My Drive/laduma/VariableDefinitions.csv')
tdf = pd.read_csv('/content/drive/My Drive/laduma/Test.csv')
df = pd.read_csv('/content/drive/My Drive/laduma/Train.csv')


In [4]:
stats = pd.read_csv('/content/drive/My Drive/laduma/game_statistics.csv')
# DROP LEAKS AS DIRECTED BY ZINDI
leaks=['Goals_scored','Goals_conceded','Action','next_action']
stats=stats.drop(leaks,axis=1)
stats.head()

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_x,next_y,event_id,next_team,next_event_id,xt_value
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,Circinus,1st half,Ryoichi,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN


**Feature Engineering**

In [5]:
teams=list(stats['Team'].unique())
print(len(teams))
teams

22


['Circinus',
 'Backward',
 'Sunflower',
 'Coma Pinwheel',
 'Eye of Sauron',
 'Butterfly',
 'Sombrero',
 'Milky Way',
 'Antennae',
 'Whirlpool',
 'Malin 1',
 'Tadpole',
 'Cosmos Redshift 7',
 'Cigar',
 'Cartwheel',
 'Fireworks',
 'Needle',
 'Sculptor',
 'Triangulum',
 'Andromeda',
 'Medusa Merger',
 'Comet']

In [6]:
teamEncoder = LabelEncoder()
teamEncoder.fit(teams)

LabelEncoder()

In [7]:
stats['Team'] = teamEncoder.transform(stats['Team']).astype('int')


In [8]:
stats['no_of_managers']=stats.groupby('Team')['Manager'].transform('nunique')
stats['no_of_teams']=stats.groupby('Manager')['Team'].transform('nunique')

le=LabelEncoder()
stats.Manager=le.fit_transform(stats.Manager).astype('float')
stats.head()

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_x,next_y,event_id,next_team,next_event_id,xt_value,no_of_managers,no_of_teams
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,6,1st half,52.0,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
1,ID_ZZKYYPVY,Player_O7HO1NOB,2.0,52.5,34.0,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
2,ID_ZZKYYPVY,Player_J0V7B0YE,3.0,37.0,32.7,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.05,0.38,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
3,ID_ZZKYYPVY,Player_J0V7B0YE,4.0,39.5,33.0,6,1st half,52.0,Backward,0.0,0.0,0.0,0.0,0.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0
4,ID_ZZKYYPVY,Player_J0V7B0YE,5.0,39.5,33.0,6,1st half,52.0,Backward,0.0,0.0,0.0,1.0,1.0,0.07,0.40,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0


In [9]:
stats.columns

Index(['Game_ID', 'Player_ID', 'id', 'X', 'Y', 'Team', 'Half', 'Manager',
       'Opposition_Team', 'Shots', 'SoT', 'Accurate passes',
       'Inaccurate passes', 'Passes', 'Start_minutes', 'End_minutes',
       'next_player', 'next_x', 'next_y', 'event_id', 'next_team',
       'next_event_id', 'xt_value', 'no_of_managers', 'no_of_teams'],
      dtype='object')

In [10]:
stats['GameHome']=stats['Game_ID'].astype(str)+'_'+stats['Team'].astype(str)
stats['GameAway']=stats['Game_ID'].astype(str)+'_'+stats['Team'].astype(str)

In [11]:
stats['Home Team']=stats.Team.copy()
stats['Away Team']=stats.Team.copy()

In [12]:
stats['SoF']=(stats.Shots+0.1)-(stats.SoT+0.1)
stats['dominance']=((stats.Passes>0)|(stats.Shots>0)).astype('int')
stats['duration']=stats.End_minutes-stats.Start_minutes
stats['played']=stats.dominance*stats['duration']
stats.head(1)

,Game_ID,Player_ID,id,X,Y,Team,Half,Manager,Opposition_Team,Shots,SoT,Accurate passes,Inaccurate passes,Passes,Start_minutes,End_minutes,next_player,next_x,next_y,event_id,next_team,next_event_id,xt_value,no_of_managers,no_of_teams,GameHome,GameAway,Home Team,Away Team,SoF,dominance,duration,played
0,ID_ZZKYYPVY,Player_O7HO1NOB,1.0,52.5,34.0,6,1st half,52.0,Backward,0.0,0.0,1.0,0.0,1.0,0.03,0.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9,3.0,ID_ZZKYYPVY_6,ID_ZZKYYPVY_6,6,6,0.0,1,0.34,0.34


# **EXTRACT FEATURES FOR HOME & AWAY**

In [13]:
# Group features by sum, intuitively

HomeSum=stats[['GameHome',
                 'Shots',
                 'SoT',
                 'Accurate passes','dominance',
                'Inaccurate passes','Passes',
                 'SoF',
                 'duration','played'
                ]].groupby(['GameHome']).sum().add_suffix('Sum').reset_index()
AwaySum=stats[['GameAway',
                    'Shots',
                 'SoT',
                 'Accurate passes','dominance',
                'Inaccurate passes','Passes',
                 'SoF',
                 'duration','played'
                ]].groupby(['GameAway']).sum().add_suffix('Sum').reset_index()

In [14]:
# Group features by mean; this is the most important step for high leaderboard position

HomeStats=stats[['GameHome',
                 'Shots',
                 'SoT',
                 'Accurate passes','dominance',
                'Inaccurate passes','Passes',
                 'SoF',
                 'duration','played'
                ]].groupby(['GameHome']).mean().reset_index()
HomeStats=pd.merge(HomeStats,HomeSum,on='GameHome',how='left')
AwayStats=stats[['GameAway',
                    'Shots',
                 'SoT',
                 'Accurate passes','dominance',
                'Inaccurate passes','Passes',
                 'SoF',
                 'duration','played'
                ]].groupby(['GameAway']).mean().reset_index()
AwayStats=pd.merge(AwayStats,AwaySum,on='GameAway',how='left')

stat_cols=list(HomeStats.columns[1:])
stat_cols

['Shots',
 'SoT',
 'Accurate passes',
 'dominance',
 'Inaccurate passes',
 'Passes',
 'SoF',
 'duration',
 'played',
 'ShotsSum',
 'SoTSum',
 'Accurate passesSum',
 'dominanceSum',
 'Inaccurate passesSum',
 'PassesSum',
 'SoFSum',
 'durationSum',
 'playedSum']

In [15]:
HomeStats['Home Team']=(HomeStats.GameHome.str.split('_',2,expand=True)[2]).astype('int')
AwayStats['Away Team']=(AwayStats.GameAway.str.split('_',2,expand=True)[2]).astype('int')

In [16]:
HomeStats.head()

,GameHome,Shots,SoT,Accurate passes,dominance,Inaccurate passes,Passes,SoF,duration,played,ShotsSum,SoTSum,Accurate passesSum,dominanceSum,Inaccurate passesSum,PassesSum,SoFSum,durationSum,playedSum,Home Team
0,ID_00EFNL7L_14,0.003681,0.000000,0.161350,0.217791,0.052761,0.214110,0.003681,0.333914,0.072571,6.0,0.0,263.0,355,86.0,349.0,6.0,544.28,118.29,14
1,ID_00EFNL7L_3,0.010405,0.003834,0.179080,0.234940,0.045455,0.224535,0.006572,0.332448,0.078138,19.0,7.0,327.0,429,83.0,410.0,12.0,607.05,142.68,3
2,ID_00J124MI_13,0.005340,0.001335,0.222964,0.262127,0.033823,0.256787,0.004005,0.332875,0.087219,12.0,3.0,501.0,589,76.0,577.0,9.0,747.97,195.98,13
3,ID_00J124MI_2,0.011146,0.003715,0.160372,0.215480,0.043963,0.204334,0.007430,0.333115,0.071771,18.0,6.0,259.0,348,71.0,330.0,12.0,537.98,115.91,2
4,ID_03VRW8WV_1,0.002674,0.001604,0.205882,0.255615,0.047059,0.252941,0.001070,0.334112,0.085123,5.0,3.0,385.0,478,88.0,473.0,2.0,624.79,159.18,1


In [17]:
TeamHome=HomeStats.groupby('Home Team').mean().reset_index()
TeamAway=AwayStats.groupby('Away Team').mean().reset_index()

In [18]:
TeamHome=TeamHome.add_suffix(' mean')
TeamAway=TeamAway.add_suffix(' mean')
mean_cols=list(TeamHome.columns[1:])
mean_cols

['Shots mean',
 'SoT mean',
 'Accurate passes mean',
 'dominance mean',
 'Inaccurate passes mean',
 'Passes mean',
 'SoF mean',
 'duration mean',
 'played mean',
 'ShotsSum mean',
 'SoTSum mean',
 'Accurate passesSum mean',
 'dominanceSum mean',
 'Inaccurate passesSum mean',
 'PassesSum mean',
 'SoFSum mean',
 'durationSum mean',
 'playedSum mean']

In [19]:
TeamHome=TeamHome.rename({'Home Team mean':'Home Team'},axis=1)
TeamAway=TeamAway.rename({'Away Team mean':'Away Team'},axis=1)

In [20]:
HomeManager=stats[['GameHome','Manager','no_of_managers','no_of_teams']].drop_duplicates('GameHome')
AwayManager=stats[['GameAway','Manager','no_of_managers','no_of_teams']].drop_duplicates('GameAway')

In [21]:
stat_cols=stat_cols+['no_of_managers','no_of_teams']

In [22]:
tdf['Score']=np.nan
tdf

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score
0,2018-12-08,3,102.0,ID_8518U587,Antennae,Cosmos Redshift 7,NaN
1,2018-09-11,3,18.0,ID_H49BIKG7,Cosmos Redshift 7,Tadpole,NaN
2,2018-09-25,3,32.0,ID_PO6SP4VA,Cosmos Redshift 7,Medusa Merger,NaN
3,2018-10-02,3,40.0,ID_MZRCNBAQ,Cartwheel,Cosmos Redshift 7,NaN
4,2018-12-20,3,123.0,ID_CV9VOLIU,Cosmos Redshift 7,Needle,NaN
...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,Malin 1,Needle,NaN
230,2018-12-11,3,300.0,ID_172V2IXW,Tadpole,Needle,NaN
231,2018-12-22,3,300.0,ID_2ZK5TE39,Circinus,Malin 1,NaN
232,2019-05-07,3,300.0,ID_IS54QGW9,Needle,Andromeda,NaN


In [23]:
#append train to test set for preprocessing
df=df.append(tdf[df.columns])
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score
0,2017-02-15,1,1.0,ID_KAG4KAE9,Antennae,Andromeda,Away win
1,2016-09-14,1,12.0,ID_T7H1XN5Z,Andromeda,Antennae,Draw
2,2016-12-12,1,13.0,ID_O1P9FGN4,Andromeda,Butterfly,Away win
3,2017-01-25,1,14.0,ID_TR5M4KQV,Andromeda,Cigar,Away win
4,2016-08-24,1,15.0,ID_EOIECRE2,Andromeda,Circinus,Home Win
...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,Malin 1,Needle,NaN
230,2018-12-11,3,300.0,ID_172V2IXW,Tadpole,Needle,NaN
231,2018-12-22,3,300.0,ID_2ZK5TE39,Circinus,Malin 1,NaN
232,2019-05-07,3,300.0,ID_IS54QGW9,Needle,Andromeda,NaN


In [24]:
df['Home Team'] = teamEncoder.transform(df['Home Team']).astype('int')
df['Away Team'] = teamEncoder.transform(df['Away Team']).astype('int')
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score
0,2017-02-15,1,1.0,ID_KAG4KAE9,1,0,Away win
1,2016-09-14,1,12.0,ID_T7H1XN5Z,0,1,Draw
2,2016-12-12,1,13.0,ID_O1P9FGN4,0,3,Away win
3,2017-01-25,1,14.0,ID_TR5M4KQV,0,5,Away win
4,2016-08-24,1,15.0,ID_EOIECRE2,0,6,Home Win
...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,12,15,NaN
230,2018-12-11,3,300.0,ID_172V2IXW,19,15,NaN
231,2018-12-22,3,300.0,ID_2ZK5TE39,6,12,NaN
232,2019-05-07,3,300.0,ID_IS54QGW9,15,0,NaN


In [25]:
GameHome1=df.copy()
GameHome2=df.copy()
GameHome1['GameHome']=GameHome1['Game_ID'].astype(str)+'_'+GameHome1['Home Team'].astype(str)
GameHome2['GameHome']=GameHome2['Game_ID'].astype(str)+'_'+GameHome2['Away Team'].astype(str)
GameHome=GameHome1[['GameHome','Season']].append(GameHome2[['GameHome','Season']])
GameHome=GameHome.drop_duplicates(['GameHome','Season'])

In [26]:
df['Min Team'] = df[['Home Team','Away Team']].min(axis=1)
df['Max Team'] = df[['Home Team','Away Team']].max(axis=1)
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,Min Team,Max Team
0,2017-02-15,1,1.0,ID_KAG4KAE9,1,0,Away win,0,1
1,2016-09-14,1,12.0,ID_T7H1XN5Z,0,1,Draw,0,1
2,2016-12-12,1,13.0,ID_O1P9FGN4,0,3,Away win,0,3
3,2017-01-25,1,14.0,ID_TR5M4KQV,0,5,Away win,0,5
4,2016-08-24,1,15.0,ID_EOIECRE2,0,6,Home Win,0,6
...,...,...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,12,15,NaN,12,15
230,2018-12-11,3,300.0,ID_172V2IXW,19,15,NaN,15,19
231,2018-12-22,3,300.0,ID_2ZK5TE39,6,12,NaN,6,12
232,2019-05-07,3,300.0,ID_IS54QGW9,15,0,NaN,0,15


In [27]:
df['H']=df['Home Team'].copy()
df['A']=df['Away Team'].copy()
df['MinisHome']=(df['H']==df['Min Team']).astype('int')
df['Home Team']=df['Min Team'].copy()
df['Away Team'] = df['Max Team'].copy()

In [28]:
df.drop(['Min Team','Max Team'],axis=1,inplace=True)

In [29]:
df['GameHome']=df['Game_ID'].astype(str)+'_'+df['Home Team'].astype(str)
df['GameAway']=df['Game_ID'].astype(str)+'_'+df['Away Team'].astype(str)

In [30]:
df['Score'] = df['Score'].replace({'Draw':1,'Away win':0, 'Home Win':2})
df.Score[df.MinisHome==0]=(df.Score-2).abs()
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,H,A,MinisHome,GameHome,GameAway
0,2017-02-15,1,1.0,ID_KAG4KAE9,0,1,2.0,1,0,0,ID_KAG4KAE9_0,ID_KAG4KAE9_1
1,2016-09-14,1,12.0,ID_T7H1XN5Z,0,1,1.0,0,1,1,ID_T7H1XN5Z_0,ID_T7H1XN5Z_1
2,2016-12-12,1,13.0,ID_O1P9FGN4,0,3,0.0,0,3,1,ID_O1P9FGN4_0,ID_O1P9FGN4_3
3,2017-01-25,1,14.0,ID_TR5M4KQV,0,5,0.0,0,5,1,ID_TR5M4KQV_0,ID_TR5M4KQV_5
4,2016-08-24,1,15.0,ID_EOIECRE2,0,6,2.0,0,6,1,ID_EOIECRE2_0,ID_EOIECRE2_6
...,...,...,...,...,...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,12,15,NaN,12,15,1,ID_QKQ00O16_12,ID_QKQ00O16_15
230,2018-12-11,3,300.0,ID_172V2IXW,15,19,NaN,19,15,0,ID_172V2IXW_15,ID_172V2IXW_19
231,2018-12-22,3,300.0,ID_2ZK5TE39,6,12,NaN,6,12,1,ID_2ZK5TE39_6,ID_2ZK5TE39_12
232,2019-05-07,3,300.0,ID_IS54QGW9,0,15,NaN,15,0,0,ID_IS54QGW9_0,ID_IS54QGW9_15


In [31]:
df.Score.fillna(4,inplace=True)
df.Score = df.Score.astype('int')

## **PREPROCESS TIME FEATURES**

In [32]:
df['Date'] = pd.to_datetime(df['Date'])
df['year'] = df['Date'].dt.year
df['year']=df['year']-df['year'].min() + 1
df['month'] = df['Date'].dt.month
df['period'] = df['month']*df['year']
df['mid'] = (df.period/6).astype('int')
df['timestamp'] = (df['Date'] - df['Date'].min()).astype('str')
df['timestamp'] = df['timestamp'].str[:-5].astype('int')
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,H,A,MinisHome,GameHome,GameAway,year,month,period,mid,timestamp
0,2017-02-15,1,1.0,ID_KAG4KAE9,0,1,2,1,0,0,ID_KAG4KAE9_0,ID_KAG4KAE9_1,2,2,4,0,196
1,2016-09-14,1,12.0,ID_T7H1XN5Z,0,1,1,0,1,1,ID_T7H1XN5Z_0,ID_T7H1XN5Z_1,1,9,9,1,42
2,2016-12-12,1,13.0,ID_O1P9FGN4,0,3,0,0,3,1,ID_O1P9FGN4_0,ID_O1P9FGN4_3,1,12,12,2,131
3,2017-01-25,1,14.0,ID_TR5M4KQV,0,5,0,0,5,1,ID_TR5M4KQV_0,ID_TR5M4KQV_5,2,1,2,0,175
4,2016-08-24,1,15.0,ID_EOIECRE2,0,6,2,0,6,1,ID_EOIECRE2_0,ID_EOIECRE2_6,1,8,8,1,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,2018-12-08,3,300.0,ID_QKQ00O16,12,15,4,12,15,1,ID_QKQ00O16_12,ID_QKQ00O16_15,3,12,36,6,857
230,2018-12-11,3,300.0,ID_172V2IXW,15,19,4,19,15,0,ID_172V2IXW_15,ID_172V2IXW_19,3,12,36,6,860
231,2018-12-22,3,300.0,ID_2ZK5TE39,6,12,4,6,12,1,ID_2ZK5TE39_6,ID_2ZK5TE39_12,3,12,36,6,871
232,2019-05-07,3,300.0,ID_IS54QGW9,0,15,4,15,0,0,ID_IS54QGW9_0,ID_IS54QGW9_15,4,5,20,3,1007


In [33]:
df=df.sort_values('timestamp')
df.head()

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,H,A,MinisHome,GameHome,GameAway,year,month,period,mid,timestamp
383,2016-08-03,1,158.0,ID_WT3PHABY,13,14,2,13,14,1,ID_WT3PHABY_13,ID_WT3PHABY_14,1,8,8,1,0
389,2016-08-03,1,118.0,ID_J1010ZTT,9,16,2,9,16,1,ID_J1010ZTT_9,ID_J1010ZTT_16,1,8,8,1,0
324,2016-08-03,1,77.0,ID_IDJ1L6GL,6,11,1,6,11,1,ID_IDJ1L6GL_6,ID_IDJ1L6GL_11,1,8,8,1,0
63,2016-08-03,1,2.0,ID_Z71PIKP3,1,3,0,1,3,1,ID_Z71PIKP3_1,ID_Z71PIKP3_3,1,8,8,1,0
120,2016-08-03,1,22.0,ID_T12UUJG7,2,4,1,2,4,1,ID_T12UUJG7_2,ID_T12UUJG7_4,1,8,8,1,0


In [34]:
df['Result']=df['Score']
df=pd.get_dummies(df,columns=['Result'])
df

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,H,A,MinisHome,GameHome,GameAway,year,month,period,mid,timestamp,Result_0,Result_1,Result_2,Result_4
383,2016-08-03,1,158.0,ID_WT3PHABY,13,14,2,13,14,1,ID_WT3PHABY_13,ID_WT3PHABY_14,1,8,8,1,0,0,0,1,0
389,2016-08-03,1,118.0,ID_J1010ZTT,9,16,2,9,16,1,ID_J1010ZTT_9,ID_J1010ZTT_16,1,8,8,1,0,0,0,1,0
324,2016-08-03,1,77.0,ID_IDJ1L6GL,6,11,1,6,11,1,ID_IDJ1L6GL_6,ID_IDJ1L6GL_11,1,8,8,1,0,0,1,0,0
63,2016-08-03,1,2.0,ID_Z71PIKP3,1,3,0,1,3,1,ID_Z71PIKP3_1,ID_Z71PIKP3_3,1,8,8,1,0,1,0,0,0
120,2016-08-03,1,22.0,ID_T12UUJG7,2,4,1,2,4,1,ID_T12UUJG7_2,ID_T12UUJG7_4,1,8,8,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210,2019-05-21,3,210.0,ID_JXNNMUJK,6,18,4,6,18,1,ID_JXNNMUJK_6,ID_JXNNMUJK_18,4,5,20,3,1021,0,0,0,1
183,2019-05-21,3,213.0,ID_MWXN4B53,4,11,4,11,4,0,ID_MWXN4B53_4,ID_MWXN4B53_11,4,5,20,3,1021,0,0,0,1
37,2019-05-23,3,214.0,ID_QKWD8147,13,21,4,21,13,0,ID_QKWD8147_13,ID_QKWD8147_21,4,5,20,3,1023,0,0,0,1
104,2019-05-24,3,215.0,ID_PJ53WB0E,8,9,4,9,8,0,ID_PJ53WB0E_8,ID_PJ53WB0E_9,4,5,20,3,1024,0,0,0,1


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 646 entries, 383 to 73
Data columns (total 21 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       646 non-null    datetime64[ns]
 1   Season     646 non-null    int64         
 2   Match_ID   646 non-null    float64       
 3   Game_ID    646 non-null    object        
 4   Home Team  646 non-null    int64         
 5   Away Team  646 non-null    int64         
 6   Score      646 non-null    int64         
 7   H          646 non-null    int64         
 8   A          646 non-null    int64         
 9   MinisHome  646 non-null    int64         
 10  GameHome   646 non-null    object        
 11  GameAway   646 non-null    object        
 12  year       646 non-null    int64         
 13  month      646 non-null    int64         
 14  period     646 non-null    int64         
 15  mid        646 non-null    int64         
 16  timestamp  646 non-null    int64         
 

In [36]:
df.describe()

,Season,Match_ID,Home Team,Away Team,Score,H,A,MinisHome,year,month,period,mid,timestamp,Result_0,Result_1,Result_2,Result_4
count,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000,646.000000
mean,2.041796,115.710526,5.623839,12.770898,2.075851,9.130031,9.264706,0.507740,2.588235,6.668731,15.873065,2.303406,551.648607,0.219814,0.208978,0.208978,0.362229
std,0.825816,70.906521,4.508664,5.067925,1.592270,5.963335,6.001774,0.500327,0.928841,3.844881,9.968135,1.747881,311.749818,0.414442,0.406894,0.406894,0.481017
min,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,57.000000,2.000000,9.000000,1.000000,4.000000,4.000000,0.000000,2.000000,3.000000,8.000000,1.000000,216.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,113.000000,5.000000,13.000000,2.000000,9.000000,9.000000,1.000000,3.000000,8.000000,15.000000,2.000000,576.500000,0.000000,0.000000,0.000000,0.000000
75%,3.000000,169.000000,8.000000,17.000000,4.000000,14.000000,14.000000,1.000000,3.000000,10.000000,22.000000,3.000000,825.750000,0.000000,0.000000,0.000000,1.000000
max,3.000000,300.000000,19.000000,21.000000,4.000000,21.000000,21.000000,1.000000,4.000000,12.000000,36.000000,6.000000,1024.000000,1.000000,1.000000,1.000000,1.000000


In [37]:
target_cols=['Score','Result_0','Result_1','Result_2','Result_4']
target = 'Score'

In [38]:
df.corr().abs()['Result_0'].sort_values(ascending=False).head(20)

Result_0     1.000000
Score        0.692540
Result_4     0.400026
Season       0.375692
timestamp    0.351317
mid          0.293394
period       0.289712
year         0.288085
Result_1     0.272826
Result_2     0.272826
A            0.141233
MinisHome    0.105418
month        0.103094
Away Team    0.102948
Match_ID     0.054547
Home Team    0.047779
H            0.018528
Name: Result_0, dtype: float64

In [39]:
df.corr().abs()['Result_1'].sort_values(ascending=False).head(20)

Result_1     1.000000
Result_4     0.387361
Score        0.347559
Season       0.307487
Result_0     0.272826
Result_2     0.264188
timestamp    0.262933
year         0.227311
period       0.217829
mid          0.215728
month        0.046853
Away Team    0.011976
A            0.011596
Match_ID     0.011235
MinisHome    0.011082
Home Team    0.004041
H            0.001563
Name: Result_1, dtype: float64

In [40]:
df.corr().abs()['Result_2'].sort_values(ascending=False).head(20)

Result_2     1.000000
Result_4     0.387361
Season       0.344399
timestamp    0.329312
Result_0     0.272826
year         0.272435
period       0.270962
Result_1     0.264188
mid          0.263687
H            0.119200
Match_ID     0.092370
MinisHome    0.087238
Away Team    0.086516
month        0.085503
Home Team    0.027228
A            0.024928
Score        0.024504
Name: Result_2, dtype: float64

In [41]:
df['HomeAway']=df['Home Team'].astype(str) + '_' + df['Away Team'].astype(str)


In [42]:
df.head()

,Date,Season,Match_ID,Game_ID,Home Team,Away Team,Score,H,A,MinisHome,GameHome,GameAway,year,month,period,mid,timestamp,Result_0,Result_1,Result_2,Result_4,HomeAway
383,2016-08-03,1,158.0,ID_WT3PHABY,13,14,2,13,14,1,ID_WT3PHABY_13,ID_WT3PHABY_14,1,8,8,1,0,0,0,1,0,13_14
389,2016-08-03,1,118.0,ID_J1010ZTT,9,16,2,9,16,1,ID_J1010ZTT_9,ID_J1010ZTT_16,1,8,8,1,0,0,0,1,0,9_16
324,2016-08-03,1,77.0,ID_IDJ1L6GL,6,11,1,6,11,1,ID_IDJ1L6GL_6,ID_IDJ1L6GL_11,1,8,8,1,0,0,1,0,0,6_11
63,2016-08-03,1,2.0,ID_Z71PIKP3,1,3,0,1,3,1,ID_Z71PIKP3_1,ID_Z71PIKP3_3,1,8,8,1,0,1,0,0,0,1_3
120,2016-08-03,1,22.0,ID_T12UUJG7,2,4,1,2,4,1,ID_T12UUJG7_2,ID_T12UUJG7_4,1,8,8,1,0,0,1,0,0,2_4


In [43]:
# Garbage features

drop_cols=[
    'Date',
    'Match_ID',
    'Game_ID','GameHome','GameAway',
    'mid',
    'H',
    'A',
    'MinisHome'
] + target_cols

In [44]:
GameAway=GameHome.rename({'GameHome':'GameAway'},axis=1)
GameAway

,GameAway,Season
0,ID_KAG4KAE9_1,1
1,ID_T7H1XN5Z_0,1
2,ID_O1P9FGN4_0,1
3,ID_TR5M4KQV_0,1
4,ID_EOIECRE2_0,1
...,...,...
229,ID_QKQ00O16_15,3
230,ID_172V2IXW_15,3
231,ID_2ZK5TE39_12,3
232,ID_IS54QGW9_0,3


In [45]:
HomeStats=pd.merge(HomeStats,GameHome,on='GameHome',how='left')
AwayStats=pd.merge(AwayStats,GameAway,on='GameAway',how='left')

In [46]:
HomeStats=HomeStats.drop(['Season','Home Team'],axis=1)
AwayStats=AwayStats.drop(['Season','Away Team'],axis=1)

## **MERGE FEATURES ON DATAFRAME**

In [47]:
df=pd.merge(df,HomeManager,on='GameHome',how='left')
df=pd.merge(df,AwayManager,on='GameAway',how='left')
df=pd.merge(df,HomeStats,on='GameHome',how='left')
df=pd.merge(df,AwayStats,on='GameAway',how='left')
for col in stat_cols:
    print(col)
    df[col+'_total']=(df[col+'_x'])+(df[col+'_y'])
    df[col+'_div']=(df[col+'_x']+0.1)/(df[col+'_y']+0.1)


Shots
SoT
Accurate passes
dominance
Inaccurate passes
Passes
SoF
duration
played
ShotsSum
SoTSum
Accurate passesSum
dominanceSum
Inaccurate passesSum
PassesSum
SoFSum
durationSum
playedSum
no_of_managers
no_of_teams


In [48]:
le=LabelEncoder()
for col in ['HomeAway']:
    df[col]=le.fit_transform(df[col].astype(str)).astype('int')

# MODEL, XGBCLASSIFIER

In [49]:
cols = list(df.drop(drop_cols,axis=1).columns)

In [50]:
tdf=df[df[target]==4]
df=df[df[target]<4]
model=XGBClassifier(n_estimators=15000, learning_rate=0.23)
df=df.sort_values('timestamp')
df1=df[df.Season==1]
df2=df[df.Season==2]
df1=shuffle(df1)
model.fit(df1[cols],df1[target])
df2['pred']=model.predict(df2[cols])
print('ACCURACY ON 2ND SEASON IS: ',accuracy_score(df2[target],df2.pred))
df=shuffle(df)
model.fit(df[cols],df[target])
tdf[target]=model.predict(tdf[cols])

ACCURACY ON 2ND SEASON IS:  0.5707317073170731


In [51]:
tdf.Score.value_counts()

0    87
1    76
2    71
Name: Score, dtype: int64

In [52]:
tdf.Score[tdf.MinisHome==0]=(tdf.Score-2).abs()
tdf.Score.value_counts()

2    80
0    78
1    76
Name: Score, dtype: int64

In [53]:
sub=tdf[ss.columns]
sub.Score=sub.Score.replace({0:'Away win',1:'Draw',2:'Home Win'})
sub=pd.merge(ss[['Game_ID']],sub,on='Game_ID',how='left')
sub.to_csv('predictions.csv',index=None)
sub

,Game_ID,Score
0,ID_03VRW8WV,Draw
1,ID_05YW7A1R,Home Win
2,ID_064HV3UA,Draw
3,ID_06SV1XSM,Draw
4,ID_08N8KYTM,Home Win
...,...,...
229,ID_ZMO6O1V6,Away win
230,ID_ZQOB9ZQL,Away win
231,ID_ZRZC49M1,Draw
232,ID_ZTDZNU6I,Draw


In [54]:
sub.Score.value_counts()

Home Win    80
Away win    78
Draw        76
Name: Score, dtype: int64

In [55]:
sorted(zip(map(lambda x: round(x, 4), model.feature_importances_), cols), reverse=True)

[(0.0603, 'duration_div'),
 (0.0529, 'SoTSum_total'),
 (0.0512, 'duration_total'),
 (0.034, 'no_of_teams_div'),
 (0.0333, 'SoTSum_x'),
 (0.033, 'Accurate passesSum_y'),
 (0.0298, 'Inaccurate passes_y'),
 (0.0269, 'Shots_total'),
 (0.0269, 'ShotsSum_total'),
 (0.0267, 'duration_x'),
 (0.0262, 'Passes_x'),
 (0.0246, 'Manager_x'),
 (0.0227, 'SoFSum_total'),
 (0.0205, 'durationSum_total'),
 (0.0205, 'Manager_y'),
 (0.0185, 'dominance_x'),
 (0.0165, 'no_of_managers_div'),
 (0.0157, 'dominanceSum_x'),
 (0.0156, 'duration_y'),
 (0.0155, 'SoT_x'),
 (0.015, 'ShotsSum_x'),
 (0.0145, 'Inaccurate passesSum_div'),
 (0.0134, 'Inaccurate passesSum_y'),
 (0.0132, 'Inaccurate passes_total'),
 (0.013, 'Inaccurate passesSum_total'),
 (0.013, 'HomeAway'),
 (0.0126, 'dominance_total'),
 (0.0124, 'Inaccurate passes_div'),
 (0.0123, 'PassesSum_x'),
 (0.0115, 'Home Team'),
 (0.0115, 'Accurate passes_div'),
 (0.0114, 'Accurate passes_x'),
 (0.0113, 'SoF_total'),
 (0.0109, 'period'),
 (0.0102, 'SoTSum_div'),
 (